In [1]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
def quantity(text):
    try:
        found = re.findall('((?:[1-9][0-9]*|0)\s?(?:\/[1-9][0-9]*)?|(?:\.[1-9][0-9]*))',text)
    except AttributeError:
        found = ['1']
    found = ''.join(found)
    found = convert_to_float(found)    
    return found

def unit(text):
    try:
        found = re.findall('([0-9]|a)\s(cup|teaspoon|tablespoon|pinch|pint|gallon|quart|half-gallon|liter|ounce|pound|lb|gal|g|)(s|es|\sof)?\s',text)
    except AttributeError:
        found = ''
    try:
        found = found[0][1]
    except:
        found = text
    return found

def convert_to_float(frac_str):
    if frac_str == '':
        frac_str = '1'
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

In [3]:
url = "https://www.allrecipes.com/recipe/10275/classic-peanut-butter-cookies/"
# url = 'https://www.allrecipes.com/recipe/11683/macaroni-and-cheese-i/'

In [4]:
# page = ''
# while page == '':
#     try:
#         page = requests.get(url)
#     except:
#         print("Connection refused by the server. Retrying.")
#         time.sleep(5)
#         continue
 
# soup = BeautifulSoup(page.content, "html.parser")
soup = BeautifulSoup(open('sample.html'),'html.parser')

In [5]:
ingredient_list = [ingredient.get_text() for ingredient in soup.find_all(itemprop = "ingredients")]

ingredients_pd = pd.DataFrame({"ingredient":ingredient_list})

In [ ]:
# ingredients_pd['parsed'].tolist()
ingredients_pd['parsed'] = ingredients_pd['ingredient'].apply(
    lambda x: re.findall('((([0-9]+(\s[0-9]\/[1-9][0-9]*)?)|([0-9]\/[1-9][0-9]*))\s(((cup|teaspoon|tablespoon|pinch|pint|gallon|quart|half-gallon|liter|ounce|pound|lb|gal|g|)(s|es|\sof)?)\s)?(.*)|^.*(?! to taste))'
                         ,x)
    )
ingredients_pd['quantity'] = ingredients_pd['parsed'].apply(lambda x: x[0][0])
ingredients_pd['quantity'] = ingredients_pd['quantity'].apply(lambda x: convert_to_float(x))

ingredients_pd['unit'] = ingredients_pd['parsed'].apply(lambda x: x[0][6])
ingredients_pd['ingredient_p'] = ingredients_pd['parsed'].apply(lambda x: x[0][8])



ingredients_pd.drop(columns = ['parsed'], axis = 1, inplace=True)

In [51]:
ingredients_pd

,ingredient,parsed,quantity
0,2 cups macaroni,"[(2 cups macaroni, 2, 2, , , cups , cups, cup,...",2 cups macaroni
1,4 tablespoons butter,"[(4 tablespoons butter, 4, 4, , , tablespoons ...",4 tablespoons butter
2,"1 onion, chopped","[(1 onion, chopped, 1, 1, , , , , , , onion, c...","1 onion, chopped"
3,"1 clove garlic, minced","[(1 clove garlic, minced, 1, 1, , , , , , , cl...","1 clove garlic, minced"
4,1 (14.5 ounce) can diced tomatoes,"[(1 (14.5 ounce) can diced tomatoes, 1, 1, , ,...",1 (14.5 ounce) can diced tomatoes
5,1/2 cup milk,"[(1/2 cup milk, 1/2, , , 1/2, cup , cup, cup, ...",1/2 cup milk
6,1 cup shredded Cheddar cheese,"[(1 cup shredded Cheddar cheese, 1, 1, , , cup...",1 cup shredded Cheddar cheese
7,1/2 cup bread crumbs,"[(1/2 cup bread crumbs, 1/2, , , 1/2, cup , cu...",1/2 cup bread crumbs
8,1/2 teaspoon Italian seasoning,"[(1/2 teaspoon Italian seasoning, 1/2, , , 1/2...",1/2 teaspoon Italian seasoning
9,salt to taste,"[(salt to taste, , , , , , , , , )]",salt to taste


In [7]:
ingredients_pd['quantity'] = ingredients_pd['ingredient'].apply(lambda x: quantity(x))
ingredients_pd['unit'] = ingredients_pd['ingredient'].apply(lambda x: unit(x))

In [8]:
ingredients_pd

,ingredient,quantity,unit
0,1 cup unsalted butter,1.0,cup
1,1 cup crunchy peanut butter,1.0,cup
2,1 cup white sugar,1.0,cup
3,1 cup packed brown sugar,1.0,cup
4,2 eggs,2.0,2 eggs
5,2 1/2 cups all-purpose flour,2.5,cup
6,1 teaspoon baking powder,1.0,teaspoon
7,1/2 teaspoon salt,0.5,teaspoon
8,1 1/2 teaspoons baking soda,1.5,teaspoon


In [92]:
ingredients_pd.iloc[0,2][0][1]

'cup'